### 데이터 로드

In [1]:
import numpy as np
import pandas as pd

In [2]:
categories = pd.read_csv('./Categories.csv')
customers = pd.read_csv('./customers.csv')
discounts = pd.read_csv('./discounts.csv')
order_details = pd.read_csv('./order_details.csv')
orders = pd.read_csv('./orders.csv')
products = pd.read_csv('./Products.csv')
qna = pd.read_csv('./QnA.csv')
shoppingcart = pd.read_csv('./ShoppingCart.csv')

In [8]:
orders

,Unnamed: 0,order_id,order_region,order_region_details,order_date,payment_method,total_price,shipping_cost,item_cnt,order_name,order_tel,refundable,customer_id
0,0,R12345,서울특별시 서초구 서초1동,1641-13,2018-01-02,credit_card,62,4,1,진채원,010-1026-4651,0,L37077
1,1,R12346,서울특별시 서초구 서초3동,1451,2018-07-24,credit_card,231,11,1,강예은,010-0361-3861,1,L59173
2,2,R12347,서울특별시 서초구 서초4동 서초교대e편한세상아파트,106동 802호,2018-11-08,credit_card,298,3,2,김현우,010-7521-8436,1,L41066
3,3,R12348,서울특별시 서초구 반포1동 반포자이아파트,144동 702호,2018-04-18,credit_card,440,2,2,이정수,010-9307-0958,0,L50741
4,4,R12349,서울특별시 서초구 반포1동 삼호가든아파트,503동 208호,2018-08-13,credit_card,220,16,1,류정남,010-2851-6839,0,L53639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50099,50099,R63585,충청남도 천안시 동남구 안서동 대림e-편한세상아파트,109동 405호,2018-02-28,money_order,68,1,2,김준영,010-3149-0832,1,L76679
50100,50100,R63586,경기도 오산시 원동 대림e-편한세상2단지아파트,218동 1206호,2018-02-28,credit_card,234,1,3,권예준,010-0769-7614,0,L69577
50101,50101,R63587,충청북도 안산시 서초중앙8가,40,2018-02-28,credit_card,124,4,1,이유진,010-2579-7691,1,L74865
50102,50102,R63588,서울특별시 도봉구 쌍문1동,493,2018-02-28,credit_card,366,13,3,강영수,010-4362-2931,1,L64297


### 9.1 날짜별 매출 집계하기

In [9]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

# 주문 날짜를 기준으로 그룹화하여 집계 함수 적용
result = orders.groupby('order_date').agg(
    purchase_count=('order_date', 'count'),
    total_amount=('total_price', 'sum'),
    avg_amount=('total_price', 'mean')
).reset_index()

# order_date를 기준으로 정렬
result = result.sort_values(by='order_date')

# 결과 출력
print(result)

    order_date  purchase_count  total_amount  avg_amount
0   2018-01-01              72         28845  400.625000
1   2018-01-02              65         24775  381.153846
2   2018-01-03             103         40540  393.592233
3   2018-01-04              97         39213  404.257732
4   2018-01-05              91         33185  364.670330
..         ...             ...           ...         ...
351 2018-12-26             160         53090  331.812500
352 2018-12-27             178         77378  434.707865
353 2018-12-28             162         64552  398.469136
354 2018-12-29             136         55601  408.830882
355 2018-12-30             255         97986  384.258824

[356 rows x 4 columns]


In [11]:
result.to_csv('날짜별 매출 집계(9-1).csv', index=False)

### 9-2. 이동평균을 사용한 날짜별 추이 보기

In [12]:
orders = pd.read_csv('./orders.csv')
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [13]:
# order_date 열을 datetime 형식으로 변환
orders['order_date'] = pd.to_datetime(orders['order_date'])

# order_date 열을 인덱스로 설정
orders.set_index('order_date', inplace=True)

# 날짜별 합계 계산
result2 = orders.resample('D').sum()

# 최근 최대 7일 동안의 평균 계산
result2['seven_day_avg'] = result2['total_price'].rolling(window=7, min_periods=1).mean()

# 최근 7일 동안의 평균을 확실하게 계산
result2['seven_day_avg_strict'] = result2['total_price'].rolling(window=7, min_periods=1).apply(
    lambda x: x.mean() if pd.notnull(x).all() else None, raw=True)

# 컬럼 이름 변경
result2.rename(columns={'total_price': 'total_amount'}, inplace=True)

# 결과 확인
print(result2)

            Unnamed: 0  total_amount  shipping_cost  item_cnt  refundable  \
order_date                                                                  
2018-01-01     1838769         28845            564       185          40   
2018-01-02     1631384         24775            459       153          39   
2018-01-03     2427048         40540            692       272          52   
2018-01-04     2454254         39213            657       264          39   
2018-01-05     2217464         33185            550       225          48   
...                ...           ...            ...       ...         ...   
2018-12-26     3963627         53090           1044       353          76   
2018-12-27     4405976         77378           1106       490          88   
2018-12-28     4158410         64552           1090       390          81   
2018-12-29     3092650         55601            865       359          59   
2018-12-30     6222485         97986           1716       653         138   

C:\Users\user\AppData\Local\Temp\ipykernel_31552\4018232132.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result2 = orders.resample('D').sum()


In [15]:
# 방법 1: reset_index() 메서드 사용
result2_reset = result2.reset_index()
result2_reset = result2_reset[['order_date', 'total_amount', 'seven_day_avg', 'seven_day_avg_strict']]
print(result2_reset)

    order_date  total_amount  seven_day_avg  seven_day_avg_strict
0   2018-01-01         28845   28845.000000          28845.000000
1   2018-01-02         24775   26810.000000          26810.000000
2   2018-01-03         40540   31386.666667          31386.666667
3   2018-01-04         39213   33343.250000          33343.250000
4   2018-01-05         33185   33311.600000          33311.600000
..         ...           ...            ...                   ...
359 2018-12-26         53090   66465.142857          66465.142857
360 2018-12-27         77378   65553.857143          65553.857143
361 2018-12-28         64552   63213.285714          63213.285714
362 2018-12-29         55601   60506.000000          60506.000000
363 2018-12-30         97986   64252.000000          64252.000000

[364 rows x 4 columns]


In [16]:
result2_reset.to_csv('이동평균_날짜별 추이(9-2).csv', index=False)

### 9-3. 당월 매출 누계 구하기

In [32]:
orders = pd.read_csv('./orders.csv')
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [33]:
orders['year_month'] = orders['order_date'].dt.to_period('M')

In [34]:
# 날짜별 매출 계산
result3 = orders.groupby(['order_date', 'year_month']).agg(
    total_amount=('total_price', 'sum')
).reset_index()

In [35]:
# 월별 누계 매출 계산
result3['agg_amount'] = result3.groupby('year_month')['total_amount'].cumsum()

# 결과 출력
print(result3)

    order_date year_month  total_amount  agg_amount
0   2018-01-01    2018-01         28845       28845
1   2018-01-02    2018-01         24775       53620
2   2018-01-03    2018-01         40540       94160
3   2018-01-04    2018-01         39213      133373
4   2018-01-05    2018-01         33185      166558
..         ...        ...           ...         ...
351 2018-12-26    2018-12         53090     1663416
352 2018-12-27    2018-12         77378     1740794
353 2018-12-28    2018-12         64552     1805346
354 2018-12-29    2018-12         55601     1860947
355 2018-12-30    2018-12         97986     1958933

[356 rows x 4 columns]


In [36]:
result3.to_csv('당월 매출 누계(9-3).csv', index=False)

### 9-5. Z차트

In [37]:
orders = pd.read_csv('./orders.csv')
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [40]:
# 연도와 월별 매출 계산
orders['year'] = orders['order_date'].dt.year
orders['month'] = orders['order_date'].dt.month
daily_purchase = orders.groupby(['order_date', 'year', 'month']).agg(
    purchase_amount=('total_price', 'sum')
).reset_index()

# 월별 매출 계산
monthly_amount = daily_purchase.groupby(['year', 'month']).agg(amount=('purchase_amount', 'sum')).reset_index()

# 연도별 누계 매출 계산
monthly_amount['agg_amount'] = monthly_amount.groupby('year')['amount'].cumsum()

# 최근 12개월 평균 매출 계산
monthly_amount['year_avg_amount'] = monthly_amount['amount'].rolling(window=12, min_periods=1).mean()

# 2018년 매출만 필터링
monthly_amount_2018 = monthly_amount[monthly_amount['year'] == 2018]

# Z차트에 필요한 열만 선택
z_chart_data = monthly_amount_2018[['year', 'month', 'amount', 'agg_amount', 'year_avg_amount']]
z_chart_data['year_month'] = z_chart_data['year'].astype(str) + '-' + z_chart_data['month'].astype(str)

# 결과 출력
print(z_chart_data)

    year  month   amount  agg_amount  year_avg_amount year_month
0   2018      1   972017      972017     9.720170e+05     2018-1
1   2018      2   820721     1792738     8.963690e+05     2018-2
2   2018      3  1104937     2897675     9.658917e+05     2018-3
3   2018      4  1465386     4363061     1.090765e+06     2018-4
4   2018      5  2035664     6398725     1.279745e+06     2018-5
5   2018      6  1536137     7934862     1.322477e+06     2018-6
6   2018      7  1973261     9908123     1.415446e+06     2018-7
7   2018      8  1610582    11518705     1.439838e+06     2018-8
8   2018      9  1796125    13314830     1.479426e+06     2018-9
9   2018     10  1775258    15090088     1.509009e+06    2018-10
10  2018     11  2145704    17235792     1.566890e+06    2018-11
11  2018     12  1958933    19194725     1.599560e+06    2018-12


In [41]:
z_chart_data.to_csv('Z차트(9-5).csv', index=False)

### 10-1. 카테고리별 매출과 소계 계산   
우리는 소 카테고리 없어서 그냥 대분류로 계산

In [22]:
orders = pd.read_csv('./orders.csv')

In [51]:
# 테이블 조인
merged_df = pd.merge(orders, qna, on='customer_id')
merged_df = pd.merge(merged_df, products, on='product_id')
merged_df = pd.merge(merged_df, categories, on='category_id')

# 카테고리별 매출과 소계 계산
result5 = merged_df.groupby('category_name').agg(
    amount=('total_price', 'sum')
).reset_index()

# 소계 추가
#subtotal = result5['amount'].sum()
#result5 = result5.append({'category_name': 'subtotal', 'amount': subtotal}, ignore_index=True)

# 결과 출력
print(result5)

        category_name  amount
0  Auto & Accessories  114180
1          Electronic  176871
2             Fashion  133671
3    Home & Furniture  155208


In [52]:
result5.to_csv('카테고리별 매출(10-1).csv', index=False)

### 10-2. ABC 분석으로 잘 팔리는 상품 판별 
- 2018년 12월만

In [23]:
merged_df = pd.merge(orders, qna, on='customer_id')
merged_df = pd.merge(merged_df, products, on='product_id')
merged_df = pd.merge(merged_df, categories, on='category_id')

In [25]:
# 날짜 필터링 (2018년 12월)
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'])
filtered_df = merged_df[(merged_df['order_date'] >= '2018-12-01') & (merged_df['order_date'] <= '2018-12-31')]

# 월별 매출 계산
monthly_sales = filtered_df.groupby('category_name').agg(
    amount=('total_price', 'sum')
).reset_index()

# 합계가 높은 순으로 정렬
monthly_sales = monthly_sales.sort_values(by='amount', ascending=False)

# 구성비 계산
monthly_sales['composition_ratio'] = 100.0 * monthly_sales['amount'] / monthly_sales['amount'].sum()

# 누계 계산
monthly_sales['cumulative_ratio'] = monthly_sales['composition_ratio'].cumsum()

# ABC 등급 부여
monthly_sales['abc_rank'] = pd.cut(monthly_sales['cumulative_ratio'], bins=[0, 70, 90, 100], labels=['A', 'B', 'C'])

# 결과 출력
print(monthly_sales)

        category_name  amount  composition_ratio  cumulative_ratio abc_rank
1          Electronic   16518          29.440177         29.440177        A
3    Home & Furniture   14014          24.977276         54.417452        A
2             Fashion   13996          24.945194         79.362646        B
0  Auto & Accessories   11579          20.637354        100.000000        C


In [27]:
monthly_sales.to_csv('ABC 분석(10-2).csv', index=False)

### 10-3. 팬차트로 상품의 매출 증가율 확인하기
- 2018년 1월기준

In [59]:
merged_df = pd.merge(orders, qna, on='customer_id')
merged_df = pd.merge(merged_df, products, on='product_id')
merged_df = pd.merge(merged_df, categories, on='category_id')

In [60]:
# 날짜 변환 및 필터링 (연도 및 월 추출)
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'])
merged_df['year_month'] = merged_df['order_date'].dt.strftime('%Y-%m')
filtered_df = merged_df[['year_month', 'category_name', 'total_price']]

# 월별 카테고리별 매출 계산
monthly_category_amount = filtered_df.groupby(['year_month', 'category_name']).agg(
    amount=('total_price', 'sum')
).reset_index()

# 베이스 매출 계산
monthly_category_amount['base_amount'] = monthly_category_amount.groupby('category_name')['amount'].transform('first')

# 매출 증가율 계산
monthly_category_amount['rate'] = 100.0 * monthly_category_amount['amount'] / monthly_category_amount['base_amount']

# 결과 출력
print(monthly_category_amount.sort_values(by=['year_month', 'category_name']))

   year_month       category_name  amount  base_amount        rate
0     2018-01  Auto & Accessories    5275         5275  100.000000
1     2018-01          Electronic    7567         7567  100.000000
2     2018-01             Fashion    6653         6653  100.000000
3     2018-01    Home & Furniture    8049         8049  100.000000
4     2018-02  Auto & Accessories    7831         5275  148.454976
5     2018-02          Electronic    4266         7567   56.376371
6     2018-02             Fashion    8631         6653  129.730948
7     2018-02    Home & Furniture    5280         8049   65.598211
8     2018-03  Auto & Accessories    3078         5275   58.350711
9     2018-03          Electronic    8824         7567  116.611603
10    2018-03             Fashion    7658         6653  115.105967
11    2018-03    Home & Furniture    5952         8049   73.947074
12    2018-04  Auto & Accessories    9843         5275  186.597156
13    2018-04          Electronic   13484         7567  178.19

In [61]:
monthly_category_amount = monthly_category_amount.sort_values(by=['year_month', 'category_name'])
monthly_category_amount

,year_month,category_name,amount,base_amount,rate
0,2018-01,Auto & Accessories,5275,5275,100.000000
1,2018-01,Electronic,7567,7567,100.000000
2,2018-01,Fashion,6653,6653,100.000000
3,2018-01,Home & Furniture,8049,8049,100.000000
4,2018-02,Auto & Accessories,7831,5275,148.454976
5,2018-02,Electronic,4266,7567,56.376371
6,2018-02,Fashion,8631,6653,129.730948
7,2018-02,Home & Furniture,5280,8049,65.598211
8,2018-03,Auto & Accessories,3078,5275,58.350711
9,2018-03,Electronic,8824,7567,116.611603


In [62]:
monthly_category_amount.to_csv('팬 차트(10-3).csv', index=False)

### 10-4. 히스토그램

In [46]:
# 히스토그램 통계 계산
max_price = products['product_price'].max() + 1
min_price = products['product_price'].min()
range_price = max_price - min_price
bucket_num = 10

# 각 버킷의 범위 계산
bucket_range = range_price / bucket_num

# purchase_log_with_bucket 생성
purchase_log_with_bucket = pd.DataFrame({
    'product_price': products['product_price'],
})

# 각 값의 버킷 계산
purchase_log_with_bucket['bucket'] = ((products['product_price'] - min_price) / bucket_range).astype(int) + 1

# 결과 계산
histogram = purchase_log_with_bucket.groupby(['bucket']).agg(
    lower_limit=('product_price', 'min'),
    upper_limit=('product_price', 'max'),
    num_purchase=('product_price', 'count'),
    total_amount=('product_price', 'sum')
).reset_index()

# 각 버킷의 범위 조정
histogram['bucket_range'] = bucket_range
histogram['lower_limit'] = min_price + (histogram['bucket'] - 1) * bucket_range
histogram['upper_limit'] = histogram['lower_limit'] + bucket_range

# 결과 출력
print(histogram)


   bucket  lower_limit  upper_limit  num_purchase  total_amount  bucket_range
0       1         33.0         54.8             3           121          21.8
1       2         54.8         76.6             5           336          21.8
2       3         76.6         98.4             3           246          21.8
3       4         98.4        120.2             7           792          21.8
4       5        120.2        142.0             5           649          21.8
5       6        142.0        163.8             2           308          21.8
6       8        185.6        207.4             3           587          21.8
7       9        207.4        229.2            10          2191          21.8
8      10        229.2        251.0             4           979          21.8


In [48]:
histogram.drop(columns=['bucket_range'], inplace=True)

In [49]:
histogram

,bucket,lower_limit,upper_limit,num_purchase,total_amount
0,1,33.0,54.8,3,121
1,2,54.8,76.6,5,336
2,3,76.6,98.4,3,246
3,4,98.4,120.2,7,792
4,5,120.2,142.0,5,649
5,6,142.0,163.8,2,308
6,8,185.6,207.4,3,587
7,9,207.4,229.2,10,2191
8,10,229.2,251.0,4,979


In [50]:
histogram.to_csv('히스토그램(10-4).csv', index=False)

In [30]:
# 데이터의 분포 확인
print(purchase_log_with_bucket['bucket'].value_counts())


9     10
4      7
2      5
5      5
10     4
1      3
3      3
8      3
6      2
7      0
Name: bucket, dtype: int64


In [31]:
# 7번 구간에 값이 있는지 확인
if 7 in purchase_log_with_bucket['bucket'].unique():
    print("7번 구간에 값이 있습니다.")
else:
    print("7번 구간에 값이 없습니다.")


7번 구간에 값이 없습니다.


### 11-6. Decile 분석

In [51]:
orders = pd.read_csv('./orders.csv')

In [53]:
# user_purchase_amount 계산
user_purchase_amount = orders.groupby('customer_id').agg(
    purchase_amount=('total_price', 'sum')
).reset_index()

# 사용자 구매 금액으로 정렬하여 상위 사용자부터 decile 할당
user_purchase_amount.sort_values(by='purchase_amount', ascending=False, inplace=True)
user_purchase_amount.reset_index(drop=True, inplace=True)

# decile 할당
total_users = len(user_purchase_amount)
user_purchase_amount['decile'] = pd.qcut(user_purchase_amount.index + 1, q=10, labels=False) + 1

# decile_with_purchase_amount 계산
decile_with_purchase_amount = user_purchase_amount.groupby('decile').agg(
    amount=('purchase_amount', 'sum'),
    avg_amount=('purchase_amount', 'mean')
).reset_index()

decile_with_purchase_amount['cumulative_amount'] = decile_with_purchase_amount['amount'].cumsum()
total_amount = decile_with_purchase_amount['amount'].sum()
decile_with_purchase_amount['total_amount'] = total_amount

# 결과 계산
decile_with_purchase_amount['total_ratio'] = 100.0 * decile_with_purchase_amount['amount'] / total_amount
decile_with_purchase_amount['cumulative_ratio'] = 100.0 * decile_with_purchase_amount['cumulative_amount'] / total_amount

# 결과 출력
print(decile_with_purchase_amount)


   decile   amount   avg_amount  cumulative_amount  total_amount  total_ratio  \
0       1  5330107  1408.590645            5330107      19194725    27.768603   
1       2  3691074   975.442389            9021181      19194725    19.229627   
2       3  2754697   727.985465           11775878      19194725    14.351323   
3       4  2121532   560.658562           13897410      19194725    11.052682   
4       5  1649637   435.950581           15547047      19194725     8.594221   
5       6  1217921   321.860729           16764968      19194725     6.345082   
6       7   893141   236.030920           17658109      19194725     4.653054   
7       8   765114   202.197146           18423223      19194725     3.986064   
8       9   491852   129.982030           18915075      19194725     2.562433   
9      10   279650    73.903277           19194725      19194725     1.456911   

   cumulative_ratio  
0         27.768603  
1         46.998230  
2         61.349553  
3         72.402236 

In [54]:
decile_with_purchase_amount.drop(columns=['total_amount'], inplace=True)
decile_with_purchase_amount

,decile,amount,avg_amount,cumulative_amount,total_ratio,cumulative_ratio
0,1,5330107,1408.590645,5330107,27.768603,27.768603
1,2,3691074,975.442389,9021181,19.229627,46.998230
2,3,2754697,727.985465,11775878,14.351323,61.349553
3,4,2121532,560.658562,13897410,11.052682,72.402236
4,5,1649637,435.950581,15547047,8.594221,80.996456
5,6,1217921,321.860729,16764968,6.345082,87.341538
6,7,893141,236.030920,17658109,4.653054,91.994592
7,8,765114,202.197146,18423223,3.986064,95.980656
8,9,491852,129.982030,18915075,2.562433,98.543089
9,10,279650,73.903277,19194725,1.456911,100.000000


In [55]:
decile_with_purchase_amount.to_csv('decile(11-6).csv', index=False)

### 11-7. RFM 분석으로 사용자를 3가지 관점의 그룹으로 나누기

In [60]:
import pandas as pd
from datetime import datetime

orders = pd.read_csv('./orders.csv')

orders['order_date'] = pd.to_datetime(orders['order_date']) # 문자열을 날짜로 변환

# purchase_log 생성
purchase_log = orders.copy()
purchase_log['dt'] = purchase_log['order_date'].dt.strftime('%Y-%m-%d')

# user_rfm 생성
user_rfm = purchase_log.groupby('customer_id').agg(
    recent_date=('dt', 'max'),
    recency=('dt', lambda x: (datetime.now().date() - pd.to_datetime(x.max()).date()).days),
    frequency=('dt', 'count'),
    monetary=('total_price', 'sum')
).reset_index()

# user_rfm_rank 생성
user_rfm['r'] = pd.cut(user_rfm['recency'], bins=[-1, 14, 28, 60, 90, float('inf')], labels=[5, 4, 3, 2, 1])
user_rfm['f'] = pd.cut(user_rfm['frequency'], bins=[-1, 1, 2, 5, 10, 20, float('inf')], labels=[1, 2, 3, 4, 5, 6])
user_rfm['m'] = pd.cut(user_rfm['monetary'], bins=[-1, 400, 600, 800, 1000, float('inf')], labels=[1, 2, 3, 4, 5])

# mst_rfm_index 생성
mst_rfm_index = pd.DataFrame({'rfm_index': [1, 2, 3, 4, 5]})

# rfm_flag 생성
rfm_flag = pd.merge(mst_rfm_index.assign(key=0), user_rfm.assign(key=0), on='key').drop(columns='key')
rfm_flag['r_flag'] = (rfm_flag['rfm_index'] == rfm_flag['r']).astype(int)
rfm_flag['f_flag'] = (rfm_flag['rfm_index'] == rfm_flag['f']).astype(int)
rfm_flag['m_flag'] = (rfm_flag['rfm_index'] == rfm_flag['m']).astype(int)

# 결과 계산
result = rfm_flag.groupby('rfm_index').agg(
    r=('r_flag', 'sum'),
    f=('f_flag', 'sum'),
    m=('m_flag', 'sum')
).reset_index().sort_values(by='rfm_index', ascending=False)

# 결과 출력
print(result)

   rfm_index      r      f      m
4          5      0      0   5190
3          4      0      1   2920
2          3      0   1826   4237
1          2      0   8285   5992
0          1  37840  27727  19501


In [82]:
rfm1.to_csv('rfm(11-7-1).csv', index=False)

In [56]:
rfm = pd.read_csv('./rfm(11-7-1).csv')
rfm

,rfm_index,r,f,m
0,1,37840,27727,19501
1,2,0,8285,5992
2,3,0,1579,4237
3,4,0,216,2920
4,5,0,31,5190


1차원으로 분리

In [86]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

# purchase_log 생성
purchase_log = orders.copy()
purchase_log['dt'] = purchase_log['order_date'].dt.strftime('%Y-%m-%d')

# user_rfm 생성
user_rfm = purchase_log.groupby('customer_id').agg(
    recent_date=('dt', 'max'),
    recency=('dt', lambda x: (datetime.now().date() - pd.to_datetime(x.max()).date()).days),
    frequency=('dt', 'count'),
    monetary=('total_price', 'sum')
).reset_index()

# user_rfm_rank 생성
user_rfm['r'] = pd.cut(user_rfm['recency'], bins=[-1, 5, 10, 15, 20, float('inf')], labels=[5, 4, 3, 2, 1])
user_rfm['f'] = pd.cut(user_rfm['frequency'], bins=[-1, 1, 2, 3, 4, 5, float('inf')], labels=[1, 2, 3, 4, 5, 6])
user_rfm['m'] = pd.cut(user_rfm['monetary'], bins=[-1, 400, 600, 800, 1000, float('inf')], labels=[1, 2, 3, 4, 5])

# total_rank 계산
user_rfm['total_rank'] = user_rfm['r'].astype(int) + user_rfm['f'].astype(int) + user_rfm['m'].astype(int)

# total_rank 별 고객 수 집계
rfm2 = user_rfm.groupby('total_rank').size().reset_index(name='count')

# 결과 출력
print(rfm2)

   total_rank  count
0           3  17585
1           4   5956
2           5   4369
3           6   3215
4           7   3021
5           8   2613
6           9    877
7          10    173
8          11     29
9          12      2


In [87]:
rfm2.to_csv('rfm(11-7-2).csv', index=False)

2차원으로 사용자 인식하기

In [93]:
# 각 등급별 f 값 카운트
rfm3 = user_rfm.groupby('r').agg(
    f_5=('f', lambda x: (x == 5).sum()),
    f_4=('f', lambda x: (x == 4).sum()),
    f_3=('f', lambda x: (x == 3).sum()),
    f_2=('f', lambda x: (x == 2).sum()),
    f_1=('f', lambda x: (x == 1).sum())
).reset_index()

# r_rank 계산
rfm3['r_rank'] = 'r_' + result['r'].astype(str)

# 결과 출력
print(rfm3)

   r  f_5  f_4   f_3   f_2    f_1 r_rank
0  5    0    0     0     0      0    r_5
1  4    0    0     0     0      0    r_4
2  3    0    0     0     0      0    r_3
3  2    0    0     0     0      0    r_2
4  1   31  216  1579  8285  27727    r_1
